In [1]:
pip install ipywidgets pandas asyncpg

error: invalid-installed-package

× Cannot process installed package ipython None in '/Users/kevin/Documents/GitHub/wellan-workspace/.venv/lib/python3.13/site-packages' because it has an invalid version:
│ Invalid version: '9.9.0 2'
╰─> Starting with pip 24.1, packages with invalid versions can not be processed.

hint: To proceed this package must be uninstalled.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from ipywidgets import widgets
import pandas as pd
from io import BytesIO

# Create the file upload widget
uploader = widgets.FileUpload(
    accept='.xlsx',
    multiple=False
)

# Create an output widget to display results
output = widgets.Output()

# Global variable to store the DataFrame
df = pd.DataFrame()

# Define a function to handle file upload
def on_upload_change(change):
    global df  # Use global to make df accessible outside the function
    with output:
        output.clear_output()
        # Check if a file was uploaded
        if uploader.value:
            # Get the uploaded file (it's a tuple)
            uploaded_file = uploader.value[0]
            # Read the file content
            content = uploaded_file['content']
            # Convert to DataFrame with UTF-8 encoding
            df = pd.read_excel(BytesIO(content), engine='openpyxl')
            print("File uploaded successfully!")
            print(f"DataFrame shape: {df.shape}")
            print("\nFirst few rows:")
            print(df.head())

# Attach the handler to the upload widget
uploader.observe(on_upload_change, names='value')

# Display the widgets
display(uploader)
display(output)

# Now you can use 'df' in subsequent cells after uploading



FileUpload(value=(), accept='.xlsx', description='Upload')

Output()

In [69]:
from shared.parser import parse_excel
from pydantic import TypeAdapter
from typing import TypedDict
from datetime import datetime

upload_df = parse_excel(df)

orders = upload_df[[
    'sub',
    'main',
    'date_created',
    'customer_no',
    'creator',
    'customer',
    'dept_sales',
    'dept_fulfillment',
    'dept_shipping',
    'dept_pickup',
    'transport',
    'thermo',
    'status',
    'time_delivery_start',
    'time_delivery_end',
    'date_delivery',
    'date_shipping',
    'comment_1',
    'comment_2',
    'comment_3',
    'telephone_day',
    'mobile',
    'recipient',
    'custom',
    'last_updated'
]]

order_address = upload_df.query('transport == "黑貓宅急便"')[[
    'sub',
    'address',
    'postnumber'
]]

full_order_address = upload_df[[
    'sub',
    'address',
    'postnumber'
]]

order_items = upload_df[[
    'sub',
    'sku',
    'product',
    'qty',
    'comment'
]]
order_custom = upload_df.query('custom == True')[[

    'sub',
    'sku',
    'custom_sku',
    'custom_product',
    'custom_qty',
]]
order_tracking = upload_df.query('transport == "黑貓宅急便"')[[
    'sub'
]]

print(len(orders['sub'].drop_duplicates()))
print(len(full_order_address.drop_duplicates()))

print(len(order_address.drop_duplicates()))
print(len(order_tracking.drop_duplicates()))

print(len(orders.query('custom == True')))
print(len(order_custom))



                sub        customer creator dept_sales date_created  \
12658  O26001429001   Tara @ Taipei  網購公共帳號    9000 網購   2026-01-11   
12659  O26001429001   Tara @ Taipei  網購公共帳號    9000 網購   2026-01-11   
12660  O26001429001   Tara @ Taipei  網購公共帳號    9000 網購   2026-01-11   
12661  O26001997001             陳又菱  網購公共帳號    9000 網購   2026-01-14   
12662  O26002019001        珊(第三方登入)  網購公共帳號    9000 網購   2026-01-15   

       dept_fulfillment dept_shipping transport thermo date_delivery  ...  \
12658  0001 總倉(2F廠務辦公室)   0002 師大泰順門市      門市自取           2026-02-01  ...   
12659  0001 總倉(2F廠務辦公室)   0002 師大泰順門市      門市自取           2026-02-01  ...   
12660  0001 總倉(2F廠務辦公室)   0002 師大泰順門市      門市自取           2026-02-01  ...   
12661  0001 總倉(2F廠務辦公室)     0003 板橋門市      門市自取           2026-02-01  ...   
12662  0001 總倉(2F廠務辦公室)  0005 5號起司高鐵店      門市自取           2026-02-01  ...   

            main customer_no   dept_pickup date_shipping time_delivery_start  \
12658  O26001429     0043937  

In [5]:

import asyncio
from shared.tcat_api import *
order_address = order_address.drop_duplicates().reset_index(drop=True)

batch_size = 100
batch_list = []


for i in range(0, len(order_address), batch_size):
    batch_addresses = order_address.iloc[i : i + batch_size]
    batch_list.append(batch_addresses)

async with httpx.AsyncClient() as client:
    tasks = [Batch_Label_PostNumber(client, batch) for batch in batch_list]
    results = await asyncio.gather(*tasks, return_exceptions=True)
    

CancelledError: 

In [ ]:
result_list = []
for i in results:
    if isinstance(i, list):
        result_list.extend(i)

print(len(result_list))
# flat_list = sum(results_list, [])
postnumber_df = pd.DataFrame(result_list)
postnumber_df = postnumber_df.rename(columns = {'Search' : 'address', 'PostNumber' : 'postnumber'})


In [6]:

final_order_address = pd.merge(full_order_address.drop(columns=['postnumber']), postnumber_df.drop_duplicates(), how='left', on='address')
print(final_order_address.fillna('').drop_duplicates().reset_index(drop=True))


NameError: name 'postnumber_df' is not defined

In [70]:
from manager import *

session = DatabaseManager()

await session.connect()

try:    
    print(order_custom.head())
    # await session.upsert_dataframe(
    #     df=orders.drop_duplicates(),
    #     table_name='orders',
    #     key_columns=['sub']
    # )
    # await session.upsert_dataframe(
    #     df=full_order_address.drop_duplicates(),
    #     table_name='order_address',
    #     key_columns=['sub']
    # )
    await session.upsert_dataframe(
        df=order_custom.drop_duplicates(),
        table_name='order_custom',
        key_columns=['sub', 'sku', 'custom_sku']
    )
    # await session.upsert_dataframe(
    #     df=order_items.drop_duplicates(),
    #     table_name='order_items',
    #     key_columns=['sub', 'sku']
    # )
finally:
    # Always close the connection pool when done
    await session.close()


Database connection pool created
            sub         sku  custom_sku        custom_product custom_qty
0  O24008651002  0120000701   010400571             文山包種茶三明治餅        1.0
1  O24008651002  0120000701   010801211               薩摩芋紫薯甜餅        1.0
2  O24008651002  0120000701   140800041      心茶-包種烏龍白色戀人(2入裝)        1.0
3  O24008651002  0120000701   010901441  唐紅手信風呂敷布(夏夜藍)55x65cm        1.0
4  O24008651002  0120000701  0109005001         2025版唐紅禮盒(空盒)        1.0
Successfully upserted 2842 rows to order_custom
Database connection pool closed
